In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [2]:
from crewai import Agent, Task, Crew

In [6]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [7]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

In [12]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [10]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Creating custom tool in Crew AI

In [19]:
from crewai.tools import BaseTool

In [20]:
class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, argument: str) -> str:
        # Your tool's logic here
        return "positive"

In [21]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

In [22]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [23]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the crew

In [24]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=True,
	memory=True
)

/opt/conda/lib/python3.10/site-packages/chromadb/api/collection_configuration.py:335: DeprecationWarning: legacy embedding function config: The CHROMA_OPENAI_API_KEY environment variable is not set.
  return json.dumps(create_collection_configuration_to_json(config))
/opt/conda/lib/python3.10/site-packages/chromadb/api/rust.py:236: DeprecationWarning: legacy embedding function config
  configuration=load_collection_configuration_from_json(
/opt/conda/lib/python3.10/site-packages/chromadb/api/models/CollectionCommon.py:158: DeprecationWarning: legacy embedding function config
  return load_collection_configuration_from_json(self._model.configuration_json)


## Runnign the Crew

In [27]:
inputs = {
    "lead_name": "RedHat",
    "industry": "Computer Software Company",
    "key_decision_maker": "Mriganka Paul",
    "position": "Senior Principal Software Engineer",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: bfc5d642-d6ad-4fad-810a-161c5762d172                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: a7d65669-3421-43af-9985-fe6f5bdba240
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: a7d65669-3421-43af-9985-fe6f5bdba240
       Status: Executing Task...
    └── 🤖 Agent: Sales Representative
            Status: In Progress

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of RedHat, a company in the Computer Software Company sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


🤖 Agent: Sales Representative
    Status: In Progress

...



# Agent: Lead Sales Representative
## Final Answer: 
Subject: Collaborating for Success - Tailored Solutions for RedHat's Recent Achievements

Email Draft:

Dear Mriganka Paul,

I trust this message finds you well. I have been closely following RedHat's continued success and recent achievements, particularly in the realm of innovative technology solutions. Congratulations on the strides RedHat has made, especially in [specific mention of recent milestone or product].

At CrewAI, we appreciate and admire RedHat's dedication to fostering innovation and driving efficiency within the organization. Our tailored solutions are designed to seamlessly integrate with your current systems and enhance your operational capabilities significantly. By leveraging our innovative approach, we aim to support RedHat's commitment to excellence and continuous advancement in the tech industry.

I would welcome the opportunity to showcase how our solutions can complement RedHat's recent achievements and con

🚀 Crew: crew
├── 📋 Task: a7d65669-3421-43af-9985-fe6f5bdba240
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
└── 📋 Task: 2940ef1d-8307-4ce1-a547-3d329099e0ba
       Status: Executing Task...
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: a7d65669-3421-43af-9985-fe6f5bdba240
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
└── 📋 Task: 2940ef1d-8307-4ce1-a547-3d329099e0ba
       Assigned to: Lead Sales Representative
       Status: ✅ Completed
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 2940ef1d-8307-4ce1-a547-3d329099e0ba                                                                     │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: bfc5d642-d6ad-4fad-810a-161c5762d172                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [28]:
from IPython.display import Markdown

# Assuming 'result' is the output from your Crew execution
# You'll need to adjust this part based on how your Crew output is structured.
# The key is to get the actual Markdown string.

if hasattr(result, 'tasks_output') and isinstance(result.tasks_output, list):
    # If 'result' has a 'tasks_output' attribute that is a list of TaskOutput
    markdown_text = ""
    for task_output in result.tasks_output:
        if hasattr(task_output, 'raw'):  # Check if the TaskOutput has the 'raw' attribute.
           markdown_text += task_output.raw + "\n\n" # Append each raw output
        elif hasattr(task_output, 'summary'): # Check if the TaskOutput has the 'summary' attribute.
            markdown_text += task_output.summary + "\n\n"
    if markdown_text:
       print(markdown_text) # Print the raw markdown text
       #Markdown(markdown_text)  # Removed Markdown conversion
    else:
       print("No markdown text found in result")

elif isinstance(result, str):
  print(result) # Print if it is already a string
  #Markdown(result) # if the result is already a string
else:
    # If 'result' is not a string and doesn't have the expected structure,
    # try to convert it to a string or access the relevant attribute.
    try:
        markdown_text = str(result)
        print(markdown_text)
        #Markdown(markdown_text)
    except Exception as e:
        print(f"Error: Could not convert result to Markdown: {e}")
        print(f"Result type: {type(result)}") # print the type to help debugging

The personalized engagement strategy for RedHat should emphasize the value of a long-term strategic partnership, the seamless integration of our solutions with their current systems, and how our innovative approach can drive efficiency and innovation within their organization. This tailored approach will help align our offerings with RedHat's objectives and foster a collaborative relationship.

Subject: Collaborating for Success - Tailored Solutions for RedHat's Recent Achievements

Email Draft:

Dear Mriganka Paul,

I trust this message finds you well. I have been closely following RedHat's continued success and recent achievements, particularly in the realm of innovative technology solutions. Congratulations on the strides RedHat has made, especially in [specific mention of recent milestone or product].

At CrewAI, we appreciate and admire RedHat's dedication to fostering innovation and driving efficiency within the organization. Our tailored solutions are designed to seamlessly inte